In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize

import sklearn

### Reading in Data 

In [2]:
api = pd.read_csv('../data/api_data.csv')
imdb = pd.read_csv('../data/imdb_6k.csv')

In [4]:
imdb.shape, api.shape

((6652, 8), (6652, 11))

In [5]:
imdb.head(2)

,href,title,genre,user_plots,synopsis,director,writers,stars
0,/title/tt0111161/,The Shawshank Redemption,Drama,['Two imprisoned men bond over a number of yea...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",Director:Frank Darabont,"Writers:Stephen King(short story ""Rita Haywort...","Stars:Tim Robbins,Morgan Freeman,Bob Gunton|Se..."
1,/title/tt0068646/,The Godfather,Crime Drama,"[""An organized crime dynasty's aging patriarch...","In late summer 1945, guests are gathered for t...",Director:Francis Ford Coppola,"Writers:Mario Puzo(screenplay by),Francis Ford...","Stars:Marlon Brando,Al Pacino,James Caan|See f..."


In [6]:
api.head(2)

,title,overviews,genres,imdb_id,popularity,vote_average,vote_count,cast,director,join_director,join_cast
0,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",tt0111161,111.250,8.7,18845,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",['Frank Darabont'],FrankDarabont,TimRobbins MorganFreeman BobGunton
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",tt0068646,45.416,8.7,14225,"['Al Pacino', 'Marlon Brando', 'James Caan']",['Francis Ford Coppola'],FrancisFordCoppola,AlPacino MarlonBrando JamesCaan


In [18]:
x = api.loc[api['title']  == 'The Master of Disguise'][['overviews']]
x['overviews'][6604]

"A sweet-natured Italian waiter named Pistachio Disguisey at his father Fabbrizio's restaurant, who happens to be a member of a family with supernatural skills of disguise. But moments later the patriarch of the Disguisey family is kidnapped Fabbrizio's former arch-enemy, Devlin Bowman, a criminal mastermind in an attempt to steal the world's most precious treasures from around the world. And it's up to Pistachio to track down Bowman and save his family before Bowman kills them!"

## Creating Master DF 

In [101]:
df = api[['imdb_id','title', 'overviews', 'join_director', 'join_cast','vote_average', 'vote_count']]
df['genre'] = imdb['genre']
df['user_plots'] = imdb['user_plots']
df['synopsis'] = imdb['synopsis']

<ipython-input-101-6e11c0ad983f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre'] = imdb['genre']
<ipython-input-101-6e11c0ad983f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_plots'] = imdb['user_plots']
<ipython-input-101-6e11c0ad983f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [106]:
df.head(2)

,imdb_id,title,overviews,join_director,join_cast,vote_average,vote_count,genre,user_plots,synopsis
0,tt0111161,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,FrankDarabont,TimRobbins MorganFreeman BobGunton,8.7,18845,Drama,['Two imprisoned men bond over a number of yea...,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
1,tt0068646,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",FrancisFordCoppola,AlPacino MarlonBrando JamesCaan,8.7,14225,Crime Drama,"[""An organized crime dynasty's aging patriarch...","In late summer 1945, guests are gathered for t..."


In [107]:
df.to_csv('../data/master_df_1.csv', index=False)

## Overview Content Model 

In [109]:
overview = df['overviews']

In [110]:
tf = TfidfVectorizer(ngram_range=(1,2),
                     min_df=0, stop_words='english')
overview_matrix = tf.fit_transform(overview)

In [111]:
overview_matrix.shape

(6652, 163739)

In [114]:
cos = sklearn.metrics.pairwise.linear_kernel(overview_matrix, overview_matrix)
cos = [row for row in cos]

cos_df = pd.DataFrame(cos)
cos_df['title'] = df['title']

cos_df.set_index('title', inplace=True)
cos_df.columns = [x for x in cos_df.index]

In [115]:
cos_df.head()

,The Shawshank Redemption,The Godfather,The Dark Knight,The Godfather: Part II,12 Angry Men,The Lord of the Rings: The Return of the King,Pulp Fiction,Schindler's List,Inception,Fight Club,...,From Justin to Kelly,House of the Dead,Disaster Movie,The Hottie & The Nottie,Superbabies: Baby Geniuses 2,Manos: The Hands of Fate,Birdemic: Shock and Terror,Pledge This!,Going Overboard,Saving Christmas
title,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,1.000000,0.001995,0.000000,0.002332,0.004448,0.0,0.000000,0.000000,0.002208,0.002183,...,0.0,0.000000,0.008235,0.0,0.003710,0.000000,0.000000,0.004839,0.000000,0.000000
The Godfather,0.001995,1.000000,0.010253,0.252749,0.004582,0.0,0.012832,0.000000,0.002410,0.000000,...,0.0,0.000000,0.000000,0.0,0.005518,0.008743,0.000000,0.000000,0.000000,0.000000
The Dark Knight,0.000000,0.010253,1.000000,0.005679,0.003945,0.0,0.005896,0.005626,0.000000,0.000000,...,0.0,0.003495,0.000000,0.0,0.000000,0.000000,0.007938,0.000000,0.000000,0.004636
The Godfather: Part II,0.002332,0.252749,0.005679,1.000000,0.002939,0.0,0.007108,0.000000,0.000000,0.002639,...,0.0,0.000000,0.000000,0.0,0.000000,0.009686,0.000000,0.000000,0.003916,0.000000
12 Angry Men,0.004448,0.004582,0.003945,0.002939,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.004039,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.003753,0.009288


In [147]:
#return sorted df with the most similar movies by cos-sine sim 
def recommend_me(title):
    recs = cos_df[title]
    return recs.sort_values(ascending=False)[1:11]

In [148]:
#return index of movie in the df
def find_index(movie):    
    index = df.index
    condition = df['title'] == movie
    dex = index[condition]
    dex.tolist()
    return dex[0]

In [149]:
recommend_me('The Godfather')

title
The Godfather: Part II     0.252749
Blood Ties                 0.090632
Gotti                      0.072076
Live by Night              0.070823
Made                       0.064192
The Godfather: Part III    0.055534
The Color Purple           0.051678
Playing for Keeps          0.050906
Summer of Sam              0.050406
Warrior                    0.048586
Name: The Godfather, dtype: float64

In [153]:
df['overviews'][find_index('The Godfather')]

'Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.'

These predictions are pretty solid, however this is an easy one for the model to pick up as the overview has lots of strong words like crime, family, and specific character names

In [136]:
recommend_me('The Shining')

title
My Little Eye            0.079426
Last Action Hero         0.066326
Hotel Rwanda             0.063955
Assassination Nation     0.062834
Return to Never Land     0.062172
Blind Dating             0.049997
Adventureland            0.048518
Mr. Popper's Penguins    0.048023
Ocean's Eleven           0.045593
Name: The Shining, dtype: float64

In [154]:
df['overviews'][find_index('The Shining')]

"Jack Torrance accepts a caretaker job at the Overlook Hotel, where he, along with his wife Wendy and their son Danny, must live isolated from the rest of the world for the winter. But they aren't prepared for the madness that lurks within."

In [156]:
df['overviews'][find_index("Mr. Popper's Penguins")]

'Jim Carrey stars as Tom Popper, a successful businessman who’s clueless when it comes to the really important things in life...until he inherits six “adorable” penguins, each with its own unique personality. Soon Tom’s rambunctious roommates turn his swank New York apartment into a snowy winter wonderland — and the rest of his world upside-down.'

In [157]:
df['overviews'][find_index('Hotel Rwanda')]

'Inspired by true events, this film takes place in Rwanda in the 1990s when more than a million Tutsis were killed in a genocide that went mostly unnoticed by the rest of the world. Hotel owner Paul Rusesabagina houses over a thousand refuges in his hotel in attempt to save their lives.'

The models weakness can be seen here. Mr. Poopper's Penguins isnt similar to The Shining in the slightest, but that one word "winter" is whats matching them. This is an issue that needs to be fixed. The same can be seen for Hotel Rwanda, where its finding similarity in the word 'hotel'

## Genre + Cast + Director Model 

In [168]:
peeps = df[['title', 'join_director', 'join_cast', 'genre']]
peeps.head(2)

,title,join_director,join_cast,genre
0,The Shawshank Redemption,FrankDarabont,TimRobbins MorganFreeman BobGunton,Drama
1,The Godfather,FrancisFordCoppola,AlPacino MarlonBrando JamesCaan,Crime Drama


In [218]:
combo = []
for dex in range(6652):
        combined = ''
        for col in ['join_director', 'join_cast', 'genre']:
            combined += str(peeps[col][dex])
        combo.append(combined)

In [220]:
len(combo)

6652

In [222]:
combo = pd.DataFrame(combo, columns=['all'])

In [224]:
combo = combo['all']

In [226]:
tf = TfidfVectorizer(stop_words='english')
combo_matrix = tf.fit_transform(combo)

In [228]:
combo_matrix.shape

(6652, 9005)

In [232]:
c_cos = sklearn.metrics.pairwise.linear_kernel(combo_matrix, combo_matrix)
c_cos = [row for row in c_cos]

c_cos_df = pd.DataFrame(c_cos)
c_cos_df['title'] = df['title']

c_cos_df.set_index('title', inplace=True)
c_cos_df.columns = [x for x in c_cos_df.index]

In [233]:
def recommend_me(title):
    recs = c_cos_df[title]
    return recs.sort_values(ascending=False)[1:11]

In [234]:
recommend_me('The Shining')

title
The Two Jakes               0.318773
3 Women                     0.279991
Popeye                      0.278713
The Aristocats              0.261647
Twilight Zone: The Movie    0.253450
Wolf                        0.248512
Eyes Wide Shut              0.242638
The Witches of Eastwick     0.219971
Spartacus                   0.216418
The Departed                0.215267
Name: The Shining, dtype: float64

In [235]:
find_index('Popeye')

6055